# Kaggle Comptetition: Black Friday Sale Prediction

__Task__: The objective to predict the primary product category given other features of the product. You may also create your own features.<br>

__Metrics__: The evaluation metric for this competition is Accuracy.<br>

__Other metrics (optional)__While you are working on the problem, you should also check the precision and recall of your models. However, this is
for your learning, and will not be considered in the evaluation.<br>

__Submission Format__<br>
The solution file will be a CSV file consisting of Product_ID and your predicted class. It should contain two columns: Product_ID and Product_Category_1.<br>

_import neccessary library_

In [829]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

_Importing the data_<br>
Courtersy to Analytics Vidhya

_Exploring the data_:<br>
The data provided consists of users and the product IDs of the products they purchased. <br>
There are some other features as well. For some of the products, you will see that the target (Product_Category_1) is marked as -1. Those are your test data, i.e.the Product IDs for which you need to predict the category.<br>
Note that you will need to transform the data to a one-row per product form for this.



In [844]:
# read in the data
df = pd.read_csv('black_friday_data_kaggle.csv')


# one hot encoding the categorical data
df = pd.get_dummies(data=df, columns=['Gender', 'Age', 'Occupation',
                                   'City_Category', 'Marital_Status',
                                   'Stay_In_Current_City_Years',])
df = df.drop(['Product_Category_2', 'Product_Category_3'], axis = 1)

#drop missing data
df = df.dropna()
# print(list(df))
# print('##################')


#normalize the data
# sc = StandardScaler()
# df.loc[:, ~df.columns.isin(['User_ID', 'Product_ID', 'Product_Category_1'])] = sc.fit_transform(df.loc[:, ~df.columns.isin(['User_ID', 'Product_ID', 'Product_Category_1'])])
# # print(list(df))

In [845]:
# get the mean value of the data so that there is one one row for each product id
df = df.groupby(['Product_ID', 'Product_Category_1'])[['Gender_F', 'Gender_M', 
                                                  'Age_0-17', 'Age_18-25', 'Age_26-35','Age_36-45',
                                                  'Age_46-50', 'Age_51-55', 'Age_55+',
                                                  'Occupation_0', 'Occupation_1', 'Occupation_2',
                                                  'Occupation_3', 'Occupation_4', 'Occupation_5',
                                                  'Occupation_6', 'Occupation_7', 'Occupation_8',
                                                  'Occupation_9', 'Occupation_10', 'Occupation_11',
                                                  'Occupation_12', 'Occupation_13', 'Occupation_14',
                                                  'Occupation_15', 'Occupation_16', 'Occupation_17',
                                                  'Occupation_18', 'Occupation_19', 'Occupation_20',
                                                  'City_Category_A', 'City_Category_B', 'City_Category_C', 
                                                  'Marital_Status_0', 'Marital_Status_1',
                                                  'Stay_In_Current_City_Years_0', 'Stay_In_Current_City_Years_1',
                                                  'Stay_In_Current_City_Years_2', 'Stay_In_Current_City_Years_3',
                                                  'Stay_In_Current_City_Years_4+', 'Purchase']].mean()

df.reset_index(inplace=True)

#normalize the data
sc = StandardScaler()
df.loc[:, ~df.columns.isin(['User_ID', 'Product_ID', 'Product_Category_1'])] = sc.fit_transform(df.loc[:, ~df.columns.isin(['User_ID', 'Product_ID', 'Product_Category_1'])])

In [846]:
df.shape # (3623, 43)

(3623, 43)

In [847]:
# split the data into train and predicting set based on value of Product_Category_1
train = df.loc[df.Product_Category_1 != -1,:]
x_train = train.loc[:,~train.columns.isin(['Product_ID', 'Product_Category_1'])]
y_train = train.Product_Category_1

test = df.loc[df.Product_Category_1 == -1,:]
x_test = test.loc[:,~test.columns.isin(['Product_ID', 'Product_Category_1'])]
y_test = test.Product_Category_1

In [848]:
y_test.shape #(1207,)

(1207,)

In [849]:
'''
this is the original data set, we picked out the 
records that have product id in the test dataset
this new data provide us true target of the test set,
which were changed to -1 in the test set
'''

original= pd.read_csv('BlackFriday.csv')
original = pd.get_dummies(data=original, columns=['Gender', 'Age', 'Occupation',
                                   'City_Category', 'Marital_Status',
                                   'Stay_In_Current_City_Years',])
original = original.drop(['Product_Category_2', 'Product_Category_3'], axis = 1)
original = original.dropna()

original = original.groupby(['Product_ID', 'Product_Category_1'])[['Gender_F', 'Gender_M', 
                                                  'Age_0-17', 'Age_18-25', 'Age_26-35','Age_36-45',
                                                  'Age_46-50', 'Age_51-55', 'Age_55+',
                                                  'Occupation_0', 'Occupation_1', 'Occupation_2',
                                                  'Occupation_3', 'Occupation_4', 'Occupation_5',
                                                  'Occupation_6', 'Occupation_7', 'Occupation_8',
                                                  'Occupation_9', 'Occupation_10', 'Occupation_11',
                                                  'Occupation_12', 'Occupation_13', 'Occupation_14',
                                                  'Occupation_15', 'Occupation_16', 'Occupation_17',
                                                  'Occupation_18', 'Occupation_19', 'Occupation_20',
                                                  'City_Category_A', 'City_Category_B', 'City_Category_C', 
                                                  'Marital_Status_0', 'Marital_Status_1',
                                                  'Stay_In_Current_City_Years_0', 'Stay_In_Current_City_Years_1',
                                                  'Stay_In_Current_City_Years_2', 'Stay_In_Current_City_Years_3',
                                                  'Stay_In_Current_City_Years_4+', 'Purchase']].mean()

original.reset_index(inplace=True)


original.loc[:, ~original.columns.isin(['Product_ID', 'Product_Category_1'])] = sc.fit_transform(original.loc[:, ~original.columns.isin(['Product_ID', 'Product_Category_1'])])


x_original = original.loc[:,~original.columns.isin(['Product_ID', 'Product_Category_1'])]
y_original = original.Product_Category_1 

# real_test = original.loc[original.Product_ID.isin(test.Product_ID), ['Product_ID', 'Product_Category_1']]
real_y_test = original.loc[original.Product_ID.isin(test.Product_ID), ['Product_Category_1']]
real_x_test = original.loc[original.Product_ID.isin(test.Product_ID), ~original.columns.isin(['Product_ID', 'Product_Category_1'])]



# print(clf_rf.score(x_original, y_original))
# pred = clf_rf.predict(x_original)
# print(accuracy_score(pred, y_original))

In [853]:
# first attempt: fit the random forest
clf_rf = RandomForestClassifier(n_estimators=70,random_state=0)
clf_rf.fit(X=x_train, y=y_train)
cv_scores = cross_val_score(clf_rf, X=x_train, y=y_train, cv = 10, scoring='accuracy')
print(np.mean(cv_scores))
clf_rf.score(real_x_test, real_y_test)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


0.6176990504256631


0.6296603148301574

In [808]:
num_trees = list(range(60,80))
param_grid = dict(n_estimators = num_trees)
grid = GridSearchCV(RandomForestClassifier(), param_grid, cv = 10, return_train_score=False)
grid.fit(x_train, y_train)
rf_result = pd.DataFrame(grid.cv_results_)[['mean_test_score', 'std_test_score', 'params']]
rf_result

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


,mean_test_score,std_test_score,params
0,0.611341,0.038630,{'n_estimators': 60}
1,0.621275,0.031518,{'n_estimators': 61}
2,0.610513,0.032669,{'n_estimators': 62}
3,0.613825,0.028747,{'n_estimators': 63}
4,0.599752,0.030467,{'n_estimators': 64}
5,0.613411,0.033407,{'n_estimators': 65}
6,0.623758,0.026632,{'n_estimators': 66}
7,0.617136,0.035716,{'n_estimators': 67}
8,0.613411,0.036556,{'n_estimators': 68}
9,0.614238,0.035853,{'n_estimators': 69}


In [854]:
# predict the data and write result to csv
pred = clf_rf.predict(x_test)
result = pd.DataFrame({'Product_ID' : test.Product_ID, 'Product_Category_1' : pred})
result = result[['Product_ID', 'Product_Category_1']]
result.to_csv('Prediction.csv', index=False)
result.shape

(1207, 2)

In [871]:
# Bagging
bg = BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_features=0.8, n_estimators=50)
bg.fit(x_train, y_train)
bg.score(real_x_test, real_y_test)
# 0.6810273405136703  0.8 50

0.68019884009942

In [872]:
# predict the data and write result to csv
pred = bg.predict(x_test)
result = pd.DataFrame({'Product_ID' : test.Product_ID, 'Product_Category_1' : pred})
result = result[['Product_ID', 'Product_Category_1']]
result.to_csv('Prediction.csv', index=False)
result.shape

(1207, 2)

In [856]:
# Boosting
adb = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators=5, learning_rate=1)
adb.fit(x_train, y_train)
adb.score(real_x_test, real_y_test)

0.5666942833471417

In [857]:
# Voting 
dt = DecisionTreeClassifier()
svm = SVC(kernel = 'rbf')
evc = VotingClassifier(estimators=[('dt', dt), ('svm', svm)])
evc.fit(x_train, y_train)
evc.score(real_x_test, real_y_test)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.5890637945318973

In [784]:
clf_rf.score(real_x_test, real_y_test)

0.5244407622203812